In [33]:
import pandas as pd
import os
from collections import defaultdict

In [34]:
PATH = 'data/raw'

In [35]:
# Define column names
bus_fields = ['BusNum','Name','BaseKV','Type','Area','Zone','Owner','Vm','Va','Vmax','Vmin','Other1','Other2']
load_fields = ['BusNum','ID','Status','Area','Zone','Pd','Qd','Other1','Other2','Other3','Other4','Other5','Other6','Other7','Other8']
shunt_fields = ['BusNum','G','B','Status','Other1','Other2']
gen_fields  = ['BusNum','ID','Pg','Qg','Qmin','Qmax','Vset','Status','Pmax','Pmin',
                'Other1','Other2','Other3','Other4','Other5','Other6','Other7','Other8',
                'Other9','Other10','Other11','Other12','Other13','Other14','Other15','Other16','Other17']
branch_fields = ['FromBus','ToBus','CircuitID','R','X','B','RateA','RateB','RateC','Tap','Shift','Status','Other1','Other2']


In [36]:
def parse_section(lines, fields, delimiter=','):
    """
    Generic parser for a PSSE section.
    """
    data = []
    for line in lines:
        line = line.split('/')[0].strip()  # Remove comments
        if not line or line.startswith('0'):
            continue
        parts = [p.strip().strip("'") for p in line.split(delimiter)]
        # remove extra parsed columns
        parts = (parts[:len(fields)])
        # pad with none for missing columns
        if len(parts) < len(fields):
            parts += [None] * (len(fields) - len(parts))
        data.append(parts)
    return pd.DataFrame(data, columns=fields)

In [37]:
def parse_psse_raw(file_path, save=True):
    with open(PATH + '/' + file_path, 'r') as f:
        lines = f.readlines()

    CASE_NAME = file_path.split('.')[0]

    sections = defaultdict(list)
    current_section = 'bus'
    
    for line in lines:
        line_strip = line.strip().upper()
        # Detect section headers
        if 'BEGIN LOAD DATA' in line_strip:
            current_section = 'load'
        elif 'BEGIN FIXED SHUNT' in line_strip:
            current_section = 'shunt'
        elif 'BEGIN GENERATOR DATA' in line_strip:
            current_section = 'gen'
        elif 'BEGIN BRANCH DATA' in line_strip:
            current_section = 'branch'
        elif '0 /' in line_strip or 'END OF' in line_strip:
            current_section = None
        elif current_section:
            sections[current_section].append(line)

    # Parse sections
    bus_df = parse_section(sections.get('bus', []), bus_fields)
    load_df = parse_section(sections.get('load', []), load_fields)
    shunt_df = parse_section(sections.get('shunt', []), shunt_fields)
    gen_df = parse_section(sections.get('gen', []), gen_fields)
    branch_df = parse_section(sections.get('branch', []), branch_fields)

    res =  {
        'bus': bus_df,
        'load': load_df,
        'shunt': shunt_df,
        'gen': gen_df,
        'branch': branch_df
    }

    if save:
        os.makedirs(f"data/{CASE_NAME}", exist_ok=True)
        for key, df in res.items():
            df.to_csv(f"data/{CASE_NAME}/{key}.csv", index=False)
    
    return res, sections

In [38]:
import os

data = os.listdir(PATH)
data

['ACTIVSg25k.RAW', 'Hawaii40_20231026.RAW', 'Texas7k_20210804.RAW']

In [39]:
a,b = parse_psse_raw(data[1])

In [42]:
df = pd.read_csv('data/Hawaii40_20231026/bus.csv')
df.head()

,BusNum,Name,BaseKV,Type,Area,Zone,Owner,Vm,Va,Vmax,Vmin,Other1,Other2
0,1,ALOHA138,138.0,1,1,1,1,0.993545,-1.119907,1.1,0.9,1.1,0.9
1,2,ALOHA69,69.0,2,1,1,1,0.991225,-3.927372,1.1,0.9,1.1,0.9
2,3,FLOWER69,69.0,1,1,1,1,0.984548,-4.731145,1.1,0.9,1.1,0.9
3,4,WAVE69,69.0,1,1,1,1,0.978800,-5.745870,1.1,0.9,1.1,0.9
4,5,HONOLULU138,138.0,1,1,1,1,0.988985,-2.069793,1.1,0.9,1.1,0.9
